In [227]:
"""
DO NOT RUN

'Data Preparation' is only used to prepare datasets that are utilized in 'Data Visualization' & 'Data Observation'
"""

import pandas as pd
import math 
import numpy as np

'English_United States.1252'

In [5]:
#creating a list of NYC zip codes
zips_list = pd.read_csv('data/ZipCodes.txt', names=['Zip Code', 'Borough'])
ZIPLIST = list(zips_list['Zip Code'])

In [ ]:
#This code cannot be changed as it assisted in manually creating the 'BoroughZipCodes' dataset
"""
#Program to assist with manually creating a file of the zip codes contained within each NYC neighborhood        
borList = []
tempZip = []
with open("newMedianAskingRent.csv") as newMRent:
    MedRent = csv.reader(newMRent, delimiter=',')
    for line in MedRent:
        if line[2] == 'neighborhood':
            borList.append(line[0])
BORZIPLIST = {}
for bor in borList:
    tempZipstr = input("Zip Codes for " + bor + ": ")
    tempZip = tempZipstr.split(', ')
    BORZIPLIST[bor] = tempZip
    print("%s: %s" % (bor, BORZIPLIST[bor]))
    
#Write the zip code data to BoroughZipCodes.csv
with open("BoroughZipCodes.csv", "w") as BorZips:
    BorZips.write('Borough, Zip Codes\n')
    for key in BORZIPLIST.keys():
        tempWrite = ''
        first = True
        for i in BORZIPLIST[key]:
            if first:
                tempWrite += '%s'%(i)
                first = False
            else:
                tempWrite += ',%s'%(i)
        BorZips.write('%s,%s\n' % (key, tempWrite))
"""

In [7]:
#Gets all new neighborhoods that are now considered due to STOPPOINT adjustments
Nrent_dataset = pd.read_csv('data/NEW_MEDIAN_RENT.csv')
Bor_Zip_dataset = pd.read_csv('data/BoroughZipCodes.csv')
missing_neighborhoods = []
Bor_Zip_List = list(Bor_Zip_dataset.iloc[:,0])
first = True
for line in Nrent_dataset.values:
    if first:
        first = False
        continue
    neigh = line[0]
    area = line[2]
    if area != 'neighborhood':
        continue
    if neigh not in Bor_Zip_List:
        missing_neighborhoods.append(neigh)
print(missing_neighborhoods)

['Auburndale', 'East Tremont', 'Melrose', 'Morris Heights', 'Morris Park', 'North Corona', 'Oakland Gardens', 'Parkchester', 'Pelham Bay', 'Pelham Parkway', 'Richmond Hill', 'Throgs Neck', 'Wakefield', 'Whitestone', 'Williamsbridge']


In [9]:
#Checks the most important data (from 2021-2022) and if the number of missing data points is > 1/3 of the total(9),
# it is deleted from the new neighborhood list
Nrent_dataset.set_index('Unnamed: 0', inplace=True)
fail_list = []
for neigh in missing_neighborhoods:
    failcount = 0
    for num in Nrent_dataset.loc[neigh][74:]:
        if math.isnan(num):
            failcount+=1
    if failcount > (len(Nrent_dataset.loc[neigh][74:]) // 3):
        fail_list.append(neigh)
        print(f'DELETED: {neigh}, Missing Data: {failcount}')
    else:
        print(f'{neigh}, Missing Data: {failcount}')
for i in fail_list:
    missing_neighborhoods.remove(i)
print(missing_neighborhoods)

Auburndale, Missing Data: 8
East Tremont, Missing Data: 4
Melrose, Missing Data: 6
DELETED: Morris Heights, Missing Data: 11
DELETED: Morris Park, Missing Data: 9
DELETED: North Corona, Missing Data: 12
Oakland Gardens, Missing Data: 2
Parkchester, Missing Data: 6
Pelham Bay, Missing Data: 8
Pelham Parkway, Missing Data: 3
Richmond Hill, Missing Data: 0
Throgs Neck, Missing Data: 4
Wakefield, Missing Data: 4
Whitestone, Missing Data: 6
Williamsbridge, Missing Data: 4
['Auburndale', 'East Tremont', 'Melrose', 'Oakland Gardens', 'Parkchester', 'Pelham Bay', 'Pelham Parkway', 'Richmond Hill', 'Throgs Neck', 'Wakefield', 'Whitestone', 'Williamsbridge']


In [27]:
#Adds new neighborhoods that are now considered due to STOPPOINT adjustments
#Old Neighborhood/Zip Code data to layer the new data onto
"""
BOR_ZIP_DICT = Bor_Zip_dataset.to_dict()

count = 105 #starts at the highest number of the borough zip dictionary
for neigh in missing_neighborhoods:
    first = True
    count+=1
    Icount = 0
    tempZipstr = input(f"Zip Codes for {neigh}: ")
    tempZip = tempZipstr.split(', ')
    for key in BOR_ZIP_DICT.keys():
        if first:
            BOR_ZIP_DICT[key][count] = neigh
            first = False
            continue
        if Icount >= len(tempZip):
            BOR_ZIP_DICT[key][count] = math.nan
        else:
            keep_key = key
            BOR_ZIP_DICT[key][count] = tempZip[Icount]
        Icount+=1
    print(f"{neigh} Last Code: {BOR_ZIP_DICT[keep_key][count]}")
"""
#CHANGES: Results in cleaner data than before, more efficient code

'\nBOR_ZIP_DICT = Bor_Zip_dataset.to_dict()\n\ncount = 105 #starts at the highest number of the borough zip dictionary\nfor neigh in missing_neighborhoods:\n    first = True\n    count+=1\n    Icount = 0\n    tempZipstr = input(f"Zip Codes for {neigh}: ")\n    tempZip = tempZipstr.split(\', \')\n    for key in BOR_ZIP_DICT.keys():\n        if first:\n            BOR_ZIP_DICT[key][count] = neigh\n            first = False\n            continue\n        if Icount >= len(tempZip):\n            BOR_ZIP_DICT[key][count] = math.nan\n        else:\n            keep_key = key\n            BOR_ZIP_DICT[key][count] = tempZip[Icount]\n        Icount+=1\n    print(f"{neigh} Last Code: {BOR_ZIP_DICT[keep_key][count]}")\n'

In [274]:
#Save the Updated neighborhood zip code list to a csv for later access
"""
FinalBorDF = pd.DataFrame.from_dict(BOR_ZIP_DICT)
Writing_BorDF = FinalBorDF.set_index('Borough')
Writing_BorDF.to_csv('data/NEW_BOROUGH_ZIPCODES.csv')
"""

In [58]:
#List containing the future list of values under each neighborhood in "NInfoDict"
DictKey = ["Neighborhood", "Households in Neighborhood", "Percentage Representation", "Median Income", "Household Margin of Error", "Percentage Margin of Error", "Median Margin of Error", "Median Rent Price", "Expected Max Rent (30% Income)", "Expected Max Rent Margin of Error", "Actual Rent vs. Expected Max Rent"]
#CHANGES: Added more points for future full dataset, no longer need to remove any more data from income dataset

In [ ]:
#TO DO
# Overall goal is to combine the rent, income, and neighborhood information into a single dataset
# I can consolidate rent data into yearly quarters and yearly
# Income data needs to be readjusted to be based on neighborhoods instead of zip codes
# Income data also needs a dataset that is a single row (a generic income for the neighborhood)
#  This should be 2 things, for 2021 and 2022

In [119]:
#Gathering quarterly rent data from rent file and writing it to QuarterlyRentData.csv
colLen = int(((len(Nrent_dataset.columns) - 2) / 12) * 5)
rowLen = len(Nrent_dataset)
QuarterYearArray = np.empty((rowLen, colLen))
QuarterYearKeys = []
QuarterYearIdx = Nrent_dataset.index
iYear = ''
#Building the base for the yearly/quarterly datasets
for col in Nrent_dataset.columns[2:]:
    tempYear = col[0:4]
    if tempYear != iYear:
        iYear = tempYear 
        QuarterYearKeys.append(tempYear)
        for i in range(1,5):
            QuarterYearKeys.append(tempYear + '.' + str(i))
print(QuarterYearArray.shape)

Rowcount = 0
for line in Nrent_dataset.values:
    count = 0
    Colcount = 0
    for num in line[2:]:
        count+=1
        #Code to deal with NaN data
        if math.isnan(num):
            if count == 1:
                YtempVals = []
                QtempVals = []
                QLongTemp = []
            elif count in [3,6,9]:
                if len(QtempVals) == 0:
                    QLongTemp.append(num)
                else:
                    QLongTemp.append(round((sum(QtempVals) / len(QtempVals)), 2))
                    QtempVals = []
            elif count == 12:
                if len(QtempVals) == 0:
                    QLongTemp.append(num)
                else:
                    QLongTemp.append(round((sum(QtempVals) / len(QtempVals)), 2))
                if len(YtempVals) == 0:
                    QuarterYearArray[Rowcount][Colcount] = num
                else:
                    QuarterYearArray[Rowcount][Colcount] = (round((sum(YtempVals) / len(YtempVals)), 2))
                Colcount+=1
                for i in range(len(QLongTemp)):
                    QuarterYearArray[Rowcount][Colcount] = QLongTemp[i]
                    Colcount+=1
                count = 0
        else:
            #Primary data parsing code
            if count == 1:
                YtempVals = [num]
                QtempVals = [num]
                QLongTemp = []
            elif count in [3,6,9]:
                QtempVals.append(num)
                YtempVals.append(num)
                QLongTemp.append(round((sum(QtempVals) / len(QtempVals)), 2))
                QtempVals = []
            elif count == 12:
                QtempVals.append(num)
                YtempVals.append(num)
                QLongTemp.append(round((sum(QtempVals) / len(QtempVals)), 2))
                QuarterYearArray[Rowcount][Colcount] = (round((sum(YtempVals) / len(YtempVals)), 2))
                Colcount+=1
                for i in range(len(QLongTemp)):
                    QuarterYearArray[Rowcount][Colcount] = QLongTemp[i]
                    Colcount+=1
                count = 0
            else:
                QtempVals.append(num)
                YtempVals.append(num)
    Rowcount+=1
QuarterYearDF = pd.DataFrame(QuarterYearArray, index=QuarterYearIdx, columns=QuarterYearKeys)
print(QuarterYearDF)
QuarterYearDF.to_csv('data/NEW_QuarterYearlyRent.csv')
#CHANGES: Switched to work with pandas DataFrames to write the data more cleanly.

(127, 40)
                      2015   2015.1   2015.2   2015.3   2015.4     2016  \
Unnamed: 0                                                                
Astoria            2241.25  2200.00  2298.33  2266.67  2200.00  2286.17   
Auburndale         1992.00  2147.50  2030.00  1948.33  1845.00  1985.18   
Bath Beach         1815.50  1758.33  1821.00  1716.67  1966.00  2041.08   
Battery Park City  4493.17  3925.00  4432.67  4748.33  4866.67  4533.00   
Bay Ridge          1806.92  1716.33  1800.00  1849.67  1861.67  2037.50   
...                    ...      ...      ...      ...      ...      ...   
Williamsburg       3196.25  3056.67  3183.33  3293.33  3251.67  3159.83   
Windsor Terrace    2647.58  2679.00  2733.33  2668.33  2509.67  2517.42   
Woodhaven          1716.80  1700.00  1711.67  1650.00  1761.00  1656.25   
Woodside           1864.67  1991.67  1850.33  1816.67  1800.00  1835.83   
Woodstock              NaN      NaN      NaN      NaN      NaN      NaN   

              

In [256]:
#Takes in a DataFrame objects and cleans it of any commas, ignoring the indicies and column headers
def clean_commas(df):
    tempDF = df
    for idx1, line in enumerate(df.values):
        first = True
        for idx2, point in enumerate(line):
            if first:
                first = False
                continue
            tempDF.iloc[idx1,idx2] = str(point).replace(',', '')
    return tempDF

IncDF2021 = pd.read_csv('data/NEW_21MedianInc.csv')
IncDF2022 = pd.read_csv('data/NEW_22MedianInc.csv')
IncDF2021 = clean_commas(IncDF2021)
IncDF2022 = clean_commas(IncDF2022)

C:\Users\chris\AppData\Local\Temp\ipykernel_2776\293751312.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.0' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tempDF.iloc[idx1,idx2] = str(point).replace(',', '')
C:\Users\chris\AppData\Local\Temp\ipykernel_2776\293751312.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '56.0' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tempDF.iloc[idx1,idx2] = str(point).replace(',', '')
C:\Users\chris\AppData\Local\Temp\ipykernel_2776\293751312.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '896.0' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tempDF.iloc[idx1,idx2] = str(point).replace(',', '')
C

In [305]:
"""

    WORK IN PROGRESS
    (here, down)

"""
#Consolidates Zipcode based income data into neighborhood income data by taking the average of all zip codes in each neighborhood           
NInfoDict = {}
BZips = pd.read_csv('data/NEW_BOROUGH_ZIPCODES.csv')
Medianlabel = IncDF2021.columns[5][6:]
MoElabel = IncDF2021.columns[6][6:]
#Slicing the data to cut out only Median Income and Median Income Margin of Error
MedianIncFD2021 = IncDF2021.iloc[:, 5::6]
MedianIncFD2022 = IncDF2022.iloc[:, 5::6]
MedianMoEIncFD2021 = IncDF2021.iloc[:, 6::6]
MedianMoEIncFD2022 = IncDF2022.iloc[:, 6::6]

rowLen = len(IncDF2021)
colLen = len(BZips) * 2
NeighIncArray = np.empty((rowLen, colLen))
for L_idx, line in enumerate(BZips.values):
    tempInc = 0
    first = True
    lenList = np.empty(rowLen)
    for Zcode in line:
        if first:
            first = False
            continue
        if math.isnan(Zcode):
            break
        intZCode = int(Zcode)
        tag = f'{str(intZCode)} {Medianlabel}'
        if len(line) == 1:
            tempNum = pd.to_numeric(IncDF2021[tag], errors='coerce')
            tempInc = np.array(tempNum).astype(float)
            lenList.fill(1)
            break
        #Problem is that if a point has a NaN in any of the zip codes, the entire line is lost
        if isinstance(tempInc, int):
            tempInc = np.empty((len(IncDF2021), 1))
            tempNum = pd.to_numeric(IncDF2021[tag], errors='coerce')
            for idx, i in enumerate(tempNum):
                if math.isnan(i):
                    lenList[idx] = 0
                    tempInc[idx] = 0
                    continue
                lenList[idx] = 1
                tempInc[idx] = float(i)
        else:
            tempNum = pd.to_numeric(IncDF2021[tag], errors='coerce')
            #Change this to a for loop to add everything, while checking for nans
            for idx, i in enumerate(tempNum):
                if math.isnan(i):
                    continue
                lenList[idx] += 1
                tempInc[idx] += float(i)       
    for idx, _ in enumerate(tempInc):
        tempInc[idx] = tempInc[idx] / lenList[idx]
    NeighIncArray[:,L_idx] = tempInc.flatten()
print(NeighIncArray)

[[1.22767364e+005 8.84334000e+004 6.78380000e+004 ... 4.11556683e-321
  4.12050749e-321 4.12544814e-321]
 [1.33838545e+005 9.90476000e+004 7.00070000e+004 ... 7.61849226e-321
  7.64813620e-321 7.68766145e-321]
 [6.40393750e+004 5.08530000e+004 7.86980000e+004 ... 1.09583760e-320
  1.09633167e-320 1.09682573e-320]
 ...
 [1.18791636e+005 8.95766000e+004 5.13020000e+004 ... 1.50507946e-311
  6.95321438e-310 1.50490181e-311]
 [9.69827273e+004 6.67576000e+004 4.27250000e+004 ... 1.50490181e-311
  6.95321438e-310 1.50508939e-311]
 [2.02024000e+005 1.38852600e+005 8.69090000e+004 ... 1.50507740e-311
  1.50490181e-311 1.50507947e-311]]


C:\Users\chris\AppData\Local\Temp\ipykernel_2776\1641440250.py:58: RuntimeWarning: invalid value encountered in divide
  tempInc[idx] = tempInc[idx] / lenList[idx]


In [ ]:
#Used solely for testing
#Consolidating all rent monthly data into a single total estimated rent over the past 5 years
first = True
with open('newMedianAskingRent.csv') as MedAskRent:
    MedRent = csv.reader(MedAskRent, delimiter=',')
    for line in MedRent:
        begin = 0
        if first:
            first = False
        else:
            if line[2] != "neighborhood":
                continue
            borough = line[0]
            count = 0
            total = 0
            for col in line:
                if begin < 3:
                    begin += 1
                else:
                    if col == '':
                        continue
                    total += float(col)
                    count += 1
            NInfoDict[borough].append(total / count)

In [ ]:
#Used solely for testing
#Calculates the 30% income threshold for maximum recommended rent and the difference between that rent and the actual median rent
for key in NInfoDict:
    income = float(NInfoDict[key][0])
    rent = float(NInfoDict[key][4])
    MoE = float(NInfoDict[key][2][1:])
    oRentMoE = (MoE/12) * 0.3
    oRent = (income/12) * 0.3
    NInfoDict[key].append(oRent)
    NInfoDict[key].append('±' + str(oRentMoE))
    dRent = rent - oRent
    NInfoDict[key].append(dRent)

In [ ]:
#Used solely for testing
#Writes all relevant data into a single .csv file
with open('NYCIncomeRentData.csv', 'w') as FinalData:
    first = True
    for i in DictKey:
        if first:
            FinalData.write(i)
            first = False
            continue
        FinalData.write(',' + i)
    for key in NInfoDict:
        FinalData.write('\n' + key + ',')
        first = True
        for item in NInfoDict[key]:
            if first:
                FinalData.write(str(item))
                first = False
                continue
            FinalData.write(',' + str(item))

In [ ]:
#Getting monthly rent data during/post Covid(Jan 2020 and beyond) and writing it to CovidRentData.csv
colMean = []
beforeData = True
colNames = []
for i in newMData.columns:
    if i == '2020-01':
        beforeData = False
    if beforeData:
        continue
    colNames.append(i)
for col in colNames:
    count = 0
    tempList = []
    for num in newMData[col]:
        if math.isnan(num):
            continue
        tempList.append(num)
    tempList.sort()
    colMean.append(MeanCalc(tempList))
csvCovidDict = {'Year-Month': colNames, 'Mean Rent': colMean}
tempDF = pd.DataFrame(csvCovidDict)
tempRows = []
tempDF.to_csv('CovidRentData.csv')

In [ ]:
#Getting rent data divided by Boroughs and writing it to BoroughRentData.csv
BorRows = {'Bronx': [], 'Brooklyn': [], 'Manhattan': [], 'Queens': []}
areaType = newMData['areaType']
for i in range(len(areaType)):
    row1 = newMData.iloc(0)[i]
    if areaType[i] == 'borough':
        if row1[0] == 'Staten Island':
            continue
        count = 0
        tempRows = []
        for i in row1[24:-4]:#starts at 2020, ends at start of 2023
            if count > 11:
                count = 0
                BorRows[row1[0]].append(MedianCalc(tempRows))
                tempRows = []
            tempRows.append(i)
            count+=1
        BorRows[row1[0]].append(MedianCalc(tempRows))
tempDF = pd.DataFrame(BorRows.values(), BorRows.keys(), ['2020', '2021', '2022'])
tempDF.to_csv('BoroughRentData.csv')

In [ ]:
#Gathering 2021 median rent
rentDict = {}
for index, row in newMData.iterrows():
    if row[2] == "submarket" or row[2] == "borough":
        continue
    currRow = row[0]
    rentDict[currRow] = []
    for i in row[36:48]:
        if math.isnan(i):
            continue
        rentDict[currRow].append(i)
    rentDict[currRow].sort()
    rentDict[currRow] = MedianCalc(rentDict[currRow])

In [ ]:
#Coalescing and writing 2021 rent, income, and rent % data to MedianRentIncData.csv
incDict = {}
with open('NYCIncomeRentData.csv') as nycIncData:
    fullData = csv.reader(nycIncData, delimiter=',')
    first = True
    for line in fullData:
        currLine = line[0]
        if first:
            first = False
            continue
        incDict[currLine] = [float(line[1])/12]
        incDict[currLine].append(rentDict[currLine])
        incDict[currLine].append((incDict[currLine][1]/incDict[currLine][0]) * 100)
tempDF = pd.DataFrame(incDict.values(), incDict.keys(), ['Median Monthly 2021 Income', 'Median 2021 Rent', 'Actual % of Income Spent on Rent 2021'])
tempDF.to_csv('MedianRentIncData.csv')